### <span style="color:pink">Importing Libraries</span>

In [142]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from typing import List
pd.set_option('display.max_columns', None)

### <span style="color:pink">Reading Data from Source</span>

In [143]:
def load_raw_data(filename):
    BASE_DIR = Path.cwd().parents[0]
    return pd.read_csv(BASE_DIR / "data" / "raw" / filename)

In [144]:
fema = load_raw_data(r'PublicAssistanceGrantAwardActivities.csv')

In [145]:
fema.shape

(948686, 23)

In [146]:
# Datatypes of features
fema.dtypes

region                     int64
disasterNumber             int64
sriaDisaster               int64
declarationTitle          object
disasterType              object
incidentType              object
declarationDate           object
stateAbbreviation         object
state                     object
county                    object
applicantId               object
applicantName             object
pnpStatus                float64
damageCategoryCode        object
federalShareObligated    float64
dateObligated             object
pwNumber                   int64
projectTitle              object
versionNumber            float64
eligibilityStatus         object
fundingStatus             object
paCloseoutStatus          object
id                        object
dtype: object

In [147]:
fema.columns

Index(['region', 'disasterNumber', 'sriaDisaster', 'declarationTitle',
       'disasterType', 'incidentType', 'declarationDate', 'stateAbbreviation',
       'state', 'county', 'applicantId', 'applicantName', 'pnpStatus',
       'damageCategoryCode', 'federalShareObligated', 'dateObligated',
       'pwNumber', 'projectTitle', 'versionNumber', 'eligibilityStatus',
       'fundingStatus', 'paCloseoutStatus', 'id'],
      dtype='object')

In [148]:
# Instances where fema funding is more than 0
fema.loc[fema.federalShareObligated>0].head(3)

,region,disasterNumber,sriaDisaster,declarationTitle,disasterType,incidentType,declarationDate,stateAbbreviation,state,county,applicantId,applicantName,pnpStatus,damageCategoryCode,federalShareObligated,dateObligated,pwNumber,projectTitle,versionNumber,eligibilityStatus,fundingStatus,paCloseoutStatus,id
0,9,1397,0,EARTHQUAKE,DR,Earthquake,2001-12-05T00:00:00.000Z,GU,Guam,Statewide,000-U6K9Y-00,GUAM WATERWORKS AUTHORITY,0.0,F - Public Utilities,121.50,2002-02-01T00:00:00.000Z,37,PUMP STATION REPAIR - HAIRLINE CRACKS,0.0,E,O,C,756305cd-2657-42ed-bb35-8c8013c551a8
1,4,1381,0,TROPICAL STORM ALLISON,DR,Severe Storm,2001-06-17T00:00:00.000Z,FL,Florida,Washington,133-99133-00,WASHINGTON COUNTY,0.0,C - Roads and Bridges,3116.84,2001-10-03T00:00:00.000Z,210,ROADS AND BRIDGES,0.0,E,O,C,553d4c29-85ef-4002-ad7f-0b10c0b3da19
2,6,1709,0,"SEVERE STORMS, TORNADOES, AND FLOODING",DR,Severe Storm,2007-06-29T00:00:00.000Z,TX,Texas,Henderson,213-99213-00,HENDERSON (COUNTY),0.0,C - Roads and Bridges,7211.69,2007-11-09T00:00:00.000Z,2594,ROADS AND BRIDGES,0.0,E,O,C,743771d1-7292-4009-afc7-a8258ebc4414


In [149]:
# Checking for null values
fema.isna().sum().to_frame(name='Null values').sort_values(by='Null values' ,ascending=False).head(5)

,Null values
county,15300
pnpStatus,15280
versionNumber,15280
projectTitle,7495
federalShareObligated,87


In [150]:
# Checking for duplicate values
fema.duplicated(keep=False).sum()

np.int64(0)

### <span style="color:pink">1. dateObligated</span>

The dateObligated variable represents the date on which FEMA formally obligated federal funds for a Public Assistance (PA) project. In FEMA’s funding process, obligation is the official commitment of federal funds after a project has been reviewed, approved, and deemed eligible. Once funds are obligated, the associated federalShareObligated amount becomes an actual financial commitment, rather than an estimate or pending request.

In [151]:
fema.dateObligated.dtype

dtype('O')

In [152]:
fema.dateObligated.sample(10)

449634    2009-11-05T00:00:00.000Z
813929    2005-12-14T00:00:00.000Z
142479    2010-08-17T00:00:00.000Z
549434    2005-09-12T00:00:00.000Z
374787    2008-10-28T00:00:00.000Z
925       1999-12-01T00:00:00.000Z
91307     2009-12-24T00:00:00.000Z
432287    2009-01-09T00:00:00.000Z
872793    2015-12-02T00:00:00.000Z
488904    2013-11-15T00:00:00.000Z
Name: dateObligated, dtype: object

In [153]:
pd.to_datetime(fema.dateObligated.str.split("T").str[0])

0        2002-02-01
1        2001-10-03
2        2007-11-09
3        2005-01-29
4        2001-09-22
            ...    
948681   2022-01-18
948682   2022-10-17
948683   2021-03-19
948684   2022-05-18
948685   2022-06-13
Name: dateObligated, Length: 948686, dtype: datetime64[ns]

In [154]:
print(fema.dateObligated.str.split("T").str[0].min())
print(fema.dateObligated.str.split("T").str[0].max())

1998-09-15
2025-12-08


### <span style="color:pink">2. declarationDate</span>

The declarationDate variable represents the date on which FEMA officially declared a disaster or emergency for a given event. This declaration is issued by the U.S. President (or FEMA under delegated authority) after a state requests federal assistance and determines that the severity of the event exceeds state and local response capabilities. The declaration date marks the formal start of federal involvement in the disaster response and recovery process.

In [155]:
fema.declarationDate.dtype

dtype('O')

In [156]:
fema.declarationDate.sample(10)

644199    2009-12-31T00:00:00.000Z
148779    2008-03-26T00:00:00.000Z
331114    2010-07-23T00:00:00.000Z
319340    2002-07-06T00:00:00.000Z
130537    2000-02-28T00:00:00.000Z
121768    2005-10-24T00:00:00.000Z
390141    2005-10-24T00:00:00.000Z
855608    2012-07-03T00:00:00.000Z
398425    2005-10-24T00:00:00.000Z
507677    2005-10-24T00:00:00.000Z
Name: declarationDate, dtype: object

In [157]:
pd.to_datetime(fema.declarationDate.str.split("T").str[0])

0        2001-12-05
1        2001-06-17
2        2007-06-29
3        2004-09-15
4        2001-05-16
            ...    
948681   2021-09-12
948682   2021-10-24
948683   2020-04-02
948684   2021-02-19
948685   2018-10-18
Name: declarationDate, Length: 948686, dtype: datetime64[ns]

In [158]:
print(fema.declarationDate.str.split("T").str[0].min())
print(fema.declarationDate.str.split("T").str[0].max())

1998-08-26
2024-09-11


### <span style="color:pink">3. damageCategoryCode</span>

The damageCategoryCode feature represents FEMA’s official Public Assistance (PA) damage categories, which classify the type of disaster-related work or infrastructure for which federal funding was obligated. These categories are standardized across FEMA programs and play a critical role in organizing disaster response, allocating recovery funds, and assessing the nature and severity of public infrastructure damage following catastrophic events.

Each category corresponds to a specific class of disaster recovery activity, ranging from immediate emergency response to long-term infrastructure restoration and administrative management. Importantly, these categories are not merely descriptive labels—they capture structural differences in cost behavior, making them highly relevant for modeling financial risk and tail outcomes.

Below is a detailed description of each category observed in the dataset:

A – Debris Removal
This category includes costs associated with clearing disaster-generated debris such as fallen trees, rubble, and wreckage. Debris removal is often one of the earliest recovery activities and can involve significant labor, equipment, and disposal costs. While individual projects are often moderate in size, large-scale disasters (e.g., hurricanes) can lead to substantial cumulative expenditures.

B – Emergency Protective Measures
This category covers actions taken to eliminate or reduce immediate threats to public health and safety. Examples include evacuations, temporary shelters, emergency medical services, law enforcement support, and protective measures taken during or immediately after a disaster. Variants in the data (such as donated resources or emergency work) reflect different funding mechanisms. These costs are time-critical and can escalate quickly during severe events.

C – Roads and Bridges
This category involves the repair or replacement of damaged roads, bridges, and related transportation infrastructure. Because transportation networks are essential for emergency response and economic recovery, projects in this category can become very expensive, particularly when major highways or bridges are affected. Costs in this category often contribute significantly to large FEMA payouts.

D – Water Control Facilities
This category includes damage to levees, dams, flood control channels, drainage systems, and similar infrastructure. Although projects in this category are less frequent, they are often associated with extreme costs when failures occur. From a catastrophe-risk perspective, this category is especially important because it frequently drives upper-tail financial losses.

E – Buildings and Equipment
This category covers repairs or replacement of publicly owned buildings and equipment, such as schools, hospitals, government offices, and emergency facilities. Costs vary widely depending on the type and importance of the facility, and damage to critical buildings can result in large recovery expenditures.

F – Utilities
This category includes damage to public utility systems such as electricity, water supply, wastewater, and communications infrastructure. Utility failures often affect large populations and require rapid, large-scale restoration efforts, making this category a major contributor to high-cost recovery projects.

G – Parks, Recreational Facilities, and Other Items
This category encompasses parks, beaches, recreational facilities, and other non-critical public assets. While important for community recovery and quality of life, projects in this category generally involve smaller funding amounts and are less likely to drive extreme tail losses.

H – Fire Management
This category is primarily associated with wildfire events and includes firefighting activities, fire suppression infrastructure, and related management costs. In extreme wildfire scenarios, projects in this category can become highly expensive and are increasingly relevant in the context of climate-driven disaster risk.

Z – Management and Administrative Costs
This category includes state and local management costs related to administering Public Assistance grants, such as planning, coordination, compliance, and oversight. These costs do not correspond to physical damage and are typically more predictable and lower in magnitude compared to infrastructure repair categories.

From a catastrophe risk estimation standpoint, the damage category code is a crucial feature because it differentiates between routine recovery activities and high-severity infrastructure failures. Categories such as Water Control Facilities (D), Utilities (F), Roads and Bridges (C), and Fire Management (H) are more likely to generate rare but extremely high recovery costs, which dominate the upper quantiles of FEMA expenditures. In contrast, management and recreational categories tend to exhibit lower variability and smaller financial impact. This makes the damage category code particularly valuable for quantile-based modeling, where the goal is to estimate tail risk rather than average recovery costs.

In [159]:
fema.damageCategoryCode.nunique()

17

In [160]:
fema.damageCategoryCode.unique()

array(['F - Public Utilities', 'C - Roads and Bridges',
       'A - Debris Removal', 'E - Public Buildings',
       'B - Protective Measures', 'G - Recreational or Other',
       'D - Water Control Facilities', 'Z - State Management',
       'H - Fire Management', 'B - Emergency Protective Measures',
       'E - Buildings and Equipment', 'F - Utilities',
       'Z - Direct Administrative Costs',
       'B - Emergency Work Donated Resources',
       'G - Parks, Recreational Facilities, and Other Items',
       'Z - Management Costs', 'Z - Section 324 Management Costs'],
      dtype=object)

In [161]:
fema.damageCategoryCode.str.extract(r"^([A-Z])").rename(columns={0: "damage_category_code_clean"}).value_counts()

damage_category_code_clean
C                             270898
B                             217694
E                             196387
A                             103097
G                              61437
F                              58246
D                              20516
Z                              17342
H                               3069
Name: count, dtype: int64

### <span style="color:pink">4. disasterType</span>

DR — Major Disaster Declaration

A Major Disaster (DR) is declared when an event causes severe and widespread damage that exceeds the response capabilities of state and local governments. These declarations authorize a broad range of federal assistance programs, including public assistance for infrastructure repair, individual assistance, and hazard mitigation. DR events are typically associated with large-scale disasters such as major hurricanes, floods, wildfires, or earthquakes, and they account for the largest FEMA expenditures. From a catastrophe-risk perspective, DR declarations are the primary drivers of extreme tail losses.

EM — Emergency Declaration

An Emergency (EM) declaration is issued to support immediate emergency actions taken to protect lives, property, and public health. These declarations are generally more limited in scope and funding compared to major disasters and are often used for short-term response activities such as evacuations, emergency protective measures, and temporary repairs. While EM events can still involve significant costs, they are typically less severe and less expensive than DR events.

FM — Fire Management Assistance Declaration

A Fire Management (FM) declaration specifically applies to wildfire-related incidents. These declarations authorize federal assistance to help state and local governments manage and suppress fires before they escalate into major disasters. Although FM declarations are narrower in scope, extreme wildfire events can still result in substantial costs, especially in large or prolonged fire seasons.

In [162]:
fema.disasterType.unique()

array(['DR', 'EM', 'FM'], dtype=object)

In [163]:
fema.disasterType.value_counts()

disasterType
DR    909414
EM     33095
FM      6177
Name: count, dtype: int64

### <span style="color:pink">5. eligibilityStatus</span>

The eligibilityStatus feature indicates the current eligibility determination of a Public Assistance (PA) project within FEMA’s funding process. It reflects whether a project has been reviewed, approved, denied, or is still pending eligibility. Although this is an administrative field, it provides useful context about which projects ultimately receive funding and can influence observed payout patterns.

The dataset includes the following eligibility status codes:

E — Eligible
Indicates that the project has been reviewed and approved for Public Assistance funding. These projects meet FEMA’s eligibility requirements and typically proceed to funding and obligation. Most finalized and fully funded projects fall into this category.

I — Ineligible
Indicates that the project was reviewed but did not meet FEMA’s eligibility criteria. Ineligible projects generally receive no federal funding or only minimal amounts. This status is often associated with low or zero obligated costs.

P — Pending
Indicates that the project’s eligibility determination is still under review. These projects may later be approved or denied. Costs associated with pending projects can be incomplete or subject to change, making this status important to consider when interpreting financial values.

V — Partially Eligible (or Validation-related status)
Indicates that only a portion of the project is eligible for federal funding, or that the project is undergoing additional validation. These cases often result in partial reimbursements, leading to moderate funding amounts compared to fully eligible projects.

X — Withdrawn / Closed / Not Proceeding
Indicates that the project was withdrawn, canceled, or closed without funding, or otherwise determined not to proceed through the Public Assistance process. These projects typically have zero or very low obligated amounts.

In [164]:
fema.eligibilityStatus.unique()

array(['E', 'I', 'P', 'V', 'X'], dtype=object)

In [165]:
fema.eligibilityStatus.value_counts()

eligibilityStatus
E    946791
I       947
V       918
X        21
P         9
Name: count, dtype: int64

### <span style="color:pink">6. fundingStatus</span>

The fundingStatus feature indicates whether a Public Assistance (PA) project has received federal funding (obligations) or has not yet been funded. This status reflects the point a project is at in FEMA’s funding lifecycle and directly relates to whether financial amounts have been formally obligated.

O — Obligated
Indicates that FEMA has formally obligated federal funds for the project. An obligated project has completed the necessary review and approval steps, and the associated federalShareObligated amount represents actual committed funding. These records account for nearly all non-zero cost values in the dataset and are central to analyzing disaster recovery expenditures.

N — Not Obligated
Indicates that no federal funds have been obligated for the project at the time of the record. This may occur because the project is still under review, deemed ineligible, withdrawn, or awaiting additional documentation. These records typically have zero or very low obligated amounts.

In [166]:
fema.fundingStatus.unique()

array(['O', 'N'], dtype=object)

In [167]:
fema.fundingStatus.value_counts()

fundingStatus
O    946778
N      1908
Name: count, dtype: int64

### <span style="color:pink">7. incidentType</span>

In [168]:
fema.incidentType.nunique()

27

In [169]:
fema.incidentType.value_counts()

incidentType
Severe Storm         383994
Hurricane            377088
Flood                 70578
Snowstorm             36858
Severe Ice Storm      29554
Fire                  17327
Typhoon                7888
Earthquake             5862
Tornado                5836
Coastal Storm          5453
Biological             3252
Severe Storm(s)        1663
Other                  1339
Winter Storm            328
Tsunami                 321
Tropical Storm          293
Freezing                234
Mud/Landslide           223
Wildfire                187
Dam/Levee Break         164
Terrorist               128
Chemical                 55
Volcanic Eruption        50
Toxic Substances          4
Volcano                   4
Drought                   2
Snow                      1
Name: count, dtype: int64

In [170]:
incident_map = {
    # Storm-related
    "severe storm": "Severe Storm",
    "severe storm(s)": "Severe Storm",
    "tornado": "Severe Storm",

    # Winter / cold weather
    "winter storm": "Winter Storm",
    "snowstorm": "Winter Storm",
    "snow": "Winter Storm",
    "freezing": "Winter Storm",
    "severe ice storm": "Winter Storm",

    # Tropical systems
    "hurricane": "Tropical Cyclone",
    "typhoon": "Tropical Cyclone",
    "tropical storm": "Tropical Cyclone",

    # Fire
    "fire": "Fire",
    "wildfire": "Fire",

    # Flooding & coastal
    "flood": "Flood",
    "coastal storm": "Coastal Storm",
    "tsunami": "Tsunami",

    # Geological
    "earthquake": "Earthquake",
    "volcano": "Volcanic Eruption",
    "volcanic eruption": "Volcanic Eruption",
    "mud/landslide": "Landslide",
    "dam/levee break": "Infrastructure Failure",

    # Biological / chemical / human
    "biological": "Biological",
    "chemical": "Chemical",
    "toxic substances": "Chemical",
    "terrorist": "Terrorist",

    # Environmental
    "drought": "Drought",

    # Catch-all
    "other": "Other"
}

In [171]:
fema.incidentType.unique()

array(['Earthquake', 'Severe Storm', 'Hurricane', 'Flood', 'Snowstorm',
       'Severe Ice Storm', 'Typhoon', 'Fire', 'Freezing', 'Tornado',
       'Other', 'Coastal Storm', 'Dam/Levee Break', 'Terrorist',
       'Tsunami', 'Drought', 'Mud/Landslide', 'Chemical',
       'Volcanic Eruption', 'Wildfire', 'Severe Storm(s)', 'Biological',
       'Winter Storm', 'Tropical Storm', 'Volcano', 'Toxic Substances',
       'Snow'], dtype=object)

In [172]:
fema.incidentType.str.lower().map(incident_map)

0               Earthquake
1             Severe Storm
2             Severe Storm
3         Tropical Cyclone
4                    Flood
                ...       
948681                Fire
948682    Tropical Cyclone
948683          Biological
948684        Winter Storm
948685               Flood
Name: incidentType, Length: 948686, dtype: object

### <span style="color:pink">8. declarationTitle</span>

In [173]:
fema.declarationTitle.unique()

array(['EARTHQUAKE', 'TROPICAL STORM ALLISON',
       'SEVERE STORMS, TORNADOES, AND FLOODING', ..., 'Hurricane Lee',
       'Hurricane Eta', 'New Hampshire - Winter Storm'],
      shape=(1265,), dtype=object)

### <span style="color:pink">9. stateAbbreviation</span>

In [174]:
fema.stateAbbreviation.unique()

array(['GU', 'FL', 'TX', 'AL', 'MN', 'VA', 'NC', 'GA', 'ND', 'ME', 'NE',
       'KY', 'WV', 'OK', 'NH', 'OH', 'IA', 'NY', 'NJ', 'AR', 'IL', 'MO',
       'NV', 'IN', 'TN', 'KS', 'VT', 'WA', 'FM', 'SD', 'CA', 'UT', 'LA',
       'WI', 'PR', 'MS', 'PA', 'DE', 'MA', 'SC', 'MI', 'OR', 'MT', 'RI',
       'AS', 'CO', 'MD', 'AK', 'CT', 'NM', 'MP', 'AZ', 'HI', 'WY', 'VI',
       'DC', 'ID'], dtype=object)

In [175]:
fema.stateAbbreviation.nunique()

57

In [176]:
fema.stateAbbreviation.value_counts().head(10)

stateAbbreviation
FL    99952
LA    97140
NY    61386
TX    60825
MS    34612
ND    33920
IA    32637
NJ    29866
PA    28876
PR    27414
Name: count, dtype: int64

### <span style="color:pink">10. state</span>

In [177]:
fema.state.unique()

array(['Guam', 'Florida', 'Texas', 'Alabama', 'Minnesota', 'Virginia',
       'North Carolina', 'Georgia', 'North Dakota', 'Maine', 'Nebraska',
       'Kentucky', 'West Virginia', 'Oklahoma', 'New Hampshire', 'Ohio',
       'Iowa', 'New York', 'New Jersey', 'Arkansas', 'Illinois',
       'Missouri', 'Nevada', 'Indiana', 'Tennessee', 'Kansas', 'Vermont',
       'Washington', 'Federated States of Micronesia', 'South Dakota',
       'California', 'Utah', 'Louisiana', 'Wisconsin', 'Puerto Rico',
       'Mississippi', 'Pennsylvania', 'Delaware', 'Massachusetts',
       'South Carolina', 'Michigan', 'Oregon', 'Montana', 'Rhode Island',
       'American Samoa', 'Colorado', 'Maryland', 'Alaska', 'Connecticut',
       'New Mexico', 'Northern Mariana Islands', 'Arizona', 'Hawaii',
       'Wyoming', 'Virgin Islands of the U.S.', 'District of Columbia',
       'Idaho', 'Virgin Islands', 'District Of Columbia'], dtype=object)

In [178]:
fema.state.nunique()

59

In [179]:
fema.state.value_counts().head(10)

state
Florida         99952
Louisiana       97140
New York        61386
Texas           60825
Mississippi     34612
North Dakota    33920
Iowa            32637
New Jersey      29866
Pennsylvania    28876
Puerto Rico     27414
Name: count, dtype: int64

In [180]:
fema[["state", "stateAbbreviation"]].dropna().drop_duplicates().sort_values(["stateAbbreviation", "state"])

,state,stateAbbreviation
264,Alaska,AK
3,Alabama,AL
26,Arkansas,AR
211,American Samoa,AS
930,Arizona,AZ
80,California,CA
222,Colorado,CO
286,Connecticut,CT
938956,District Of Columbia,DC
3184,District of Columbia,DC


State Column Data Quality Issue

An inconsistency was identified in the state column where the same geographic entity appears under multiple textual representations. Specifically, the District of Columbia is recorded with different casing variations, and the Virgin Islands appear under two naming formats: “Virgin Islands” and “Virgin Islands of the U.S.”. These inconsistencies artificially inflate the number of unique state values and can introduce noise in exploratory analysis and downstream modeling. To resolve this, state names were standardized by normalizing casing and consolidating known duplicate representations into a single canonical form. The stateAbbreviation column remains consistent and was therefore retained as the primary geographic identifier, while the cleaned state names were preserved for reporting and visualization purposes.

In [181]:
state_fix_map = {
    # District of Columbia casing issue
    "District Of Columbia": "District of Columbia",

    # Virgin Islands variants in your data
    "Virgin Islands of the U.S.": "Virgin Islands"
}


In [182]:
fema.state.str.strip().replace(state_fix_map).nunique()

57

### <span style="color:pink">11. county</span>

The county variable represents the county-level geographic jurisdiction within a U.S. state where the Public Assistance (PA) project is located or administered. Counties are the primary local government units through which FEMA coordinates disaster recovery efforts, and they play a key role in damage assessment, project submission, and fund administration.

In the FEMA Public Assistance program, funding is typically requested and managed by local applicants such as counties, cities, school districts, or public utilities. The county field identifies the county associated with the damaged infrastructure or the applicant organization, providing a more granular geographic reference than the state alone.

From an analytical perspective, the county variable captures local exposure to disaster risk, reflecting geographic differences in hazard frequency, infrastructure density, and vulnerability. Certain counties—such as coastal counties, floodplain regions, or wildfire-prone areas—may consistently experience higher recovery costs, making this feature useful for understanding spatial patterns in disaster expenditures.

In [183]:
fema.county.unique()

array(['Statewide', 'Washington', 'Henderson', ..., 'Sutton County',
       'Robertson County', 'Rabun County'], shape=(2754,), dtype=object)

In [184]:
fema.county.nunique()

2753

### <span style="color:pink">12. projectTitle</span>

In [185]:
fema.projectTitle.unique()

array(['PUMP STATION REPAIR - HAIRLINE CRACKS', 'ROADS AND BRIDGES',
       'DEBRIS REMOVAL', ...,
       'DNREC IDA jurisdiction wide debris removal',
       'Novant UVA COVID-19 July2020', 'Barron Rd 24" Water Main'],
      shape=(397191,), dtype=object)

In [186]:
fema.projectTitle.nunique()

397190

In [187]:
fema.projectTitle.duplicated().sum()

np.int64(551495)

### <span style="color:pink">13. paCloseoutStatus</span>

The paCloseoutStatus variable indicates the final administrative status of a Public Assistance (PA) project with respect to program closeout. Closeout is the last stage in FEMA’s PA lifecycle, occurring after funds have been obligated and the project has been completed, reviewed, and reconciled. This status reflects whether FEMA has officially closed the project file.

O — Open
Indicates that the Public Assistance project is still open in FEMA’s system. An open project may have funds obligated but is still undergoing activities such as work completion, documentation review, audits, or final reconciliation. Open status does not mean the project is unfunded; it simply means the administrative closeout process is not yet complete.

C — Closed
Indicates that the project has been fully completed and formally closed out by FEMA. All eligible work has been finished, documentation has been accepted, and final funding amounts have been reconciled. Closed projects represent finalized records with no further financial changes expected.

In [188]:
fema.paCloseoutStatus.unique()

array(['C', 'O'], dtype=object)

In [189]:
fema.paCloseoutStatus.value_counts()

paCloseoutStatus
C    724251
O    224435
Name: count, dtype: int64

### <span style="color:pink">14. region</span>

The region variable identifies the FEMA administrative region responsible for coordinating disaster response and recovery activities. FEMA divides the United States into multiple regions, each overseeing disaster management across several states. This feature provides a coarse geographic grouping that captures regional differences in disaster exposure, response practices, and funding patterns. Compared to state or county, the region variable is lower in cardinality and can be useful for identifying broad spatial trends in disaster recovery costs without introducing excessive categorical complexity.

In [190]:
fema.region.unique()

array([ 9,  4,  6,  5,  3,  8,  1,  7,  2, 10])

In [191]:
fema.region.value_counts()

region
4     225528
6     213093
2     120892
7      78617
5      77388
3      66641
8      54996
1      49961
9      37405
10     24165
Name: count, dtype: int64

### <span style="color:pink">15. disasterNumber</span>

The disasterNumber is a unique identifier assigned by FEMA to each disaster declaration. All Public Assistance projects associated with the same disaster share this number. It does not carry intrinsic information about severity or cost by itself; rather, it acts as a grouping key that links multiple projects to a single disaster event. Analytically, it is useful for aggregation (e.g., total cost per disaster) and for understanding how many projects stem from a single declaration, but it should not be used directly as a predictive feature.

In [192]:
fema.disasterNumber.unique()

array([1397, 1381, 1709, ..., 4627, 4427, 4371], shape=(2301,))

In [193]:
fema.disasterNumber.duplicated().sum()

np.int64(946385)

### <span style="color:pink">16. sriaDisaster</span>

The sriaDisaster field indicates whether a disaster is classified under FEMA’s Stafford Act Section 428 (also known as SRIA – Sandy Recovery Improvement Act) procedures. SRIA disasters follow an alternative Public Assistance framework that emphasizes fixed-cost estimates and streamlined project delivery. This designation can influence how costs are estimated, obligated, and managed, making it a potentially important indicator of funding structure and cost behavior, rather than disaster severity itself.

In [194]:
fema.sriaDisaster.value_counts()

sriaDisaster
0    886543
1     62143
Name: count, dtype: int64

### <span style="color:pink">17. pnpStatus</span>

The pnpStatus variable indicates whether the applicant associated with a Public Assistance project is a Private Non-Profit (PNP) organization, such as certain hospitals, utilities, or educational institutions. PNP applicants are eligible for FEMA assistance under specific conditions and often differ from state or local governments in terms of asset types and recovery processes. This feature provides insight into the type of applicant requesting aid and can help explain variation in project costs and funding patterns.

In [195]:
fema.pnpStatus.value_counts()

pnpStatus
0.0    866136
1.0     67270
Name: count, dtype: int64

### <span style="color:pink">18. pwNumber</span>

The pwNumber is an identifier for the Project Worksheet (PW), which is the core documentation FEMA uses to define the scope, cost, and eligibility of a Public Assistance project. Each PW corresponds to a specific repair or recovery activity. While this field is critical for FEMA’s internal tracking and auditing, it functions purely as an identifier in the dataset and does not convey predictive information on its own.

In [196]:
fema.pwNumber.nunique()

20986

### <span style="color:pink">19. versionNumber</span>

The versionNumber indicates the revision count of a Project Worksheet. Projects may undergo multiple revisions as scopes change, costs are updated, or additional documentation is reviewed. Higher version numbers reflect administrative updates over time, not necessarily greater disaster severity. This feature is primarily useful for understanding data maturity and project evolution, rather than for predicting recovery costs.

In [197]:
fema.versionNumber.nunique()

57

In [198]:
fema.versionNumber.value_counts().to_frame().head(5)

,count
versionNumber,
0.0,658532
1.0,169655
2.0,58062
3.0,24304
4.0,10722


In [199]:
fema.versionNumber.value_counts().to_frame().tail(5)

,count
versionNumber,
73.0,1
74.0,1
47.0,1
65.0,1
75.0,1


In [200]:
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.loc[df.federalShareObligated > 0]

    df = df.assign(
        declarationDate=lambda x: pd.to_datetime(x.declarationDate.astype(str).str.split("T").str[0]),
        dateObligated=lambda x: pd.to_datetime(x.dateObligated.astype(str).str.split("T").str[0]),
        damageCategoryCode=lambda x: (
            x.damageCategoryCode
            .astype(str)
            .str.extract(r"^([A-Z])", expand=False)
        ),
        incidentType=lambda x: (
            x.incidentType
            .astype(str)
            .str.lower()
            .map(incident_map)
        ),
        state=lambda x: (
            x.state
            .str
            .strip()
            .replace(state_fix_map)
        )
    )

    return df


In [201]:
fema_cleaned = clean_data(fema)

In [202]:
fema_cleaned.head(5)

,region,disasterNumber,sriaDisaster,declarationTitle,disasterType,incidentType,declarationDate,stateAbbreviation,state,county,applicantId,applicantName,pnpStatus,damageCategoryCode,federalShareObligated,dateObligated,pwNumber,projectTitle,versionNumber,eligibilityStatus,fundingStatus,paCloseoutStatus,id
0,9,1397,0,EARTHQUAKE,DR,Earthquake,2001-12-05,GU,Guam,Statewide,000-U6K9Y-00,GUAM WATERWORKS AUTHORITY,0.0,F,121.50,2002-02-01,37,PUMP STATION REPAIR - HAIRLINE CRACKS,0.0,E,O,C,756305cd-2657-42ed-bb35-8c8013c551a8
1,4,1381,0,TROPICAL STORM ALLISON,DR,Severe Storm,2001-06-17,FL,Florida,Washington,133-99133-00,WASHINGTON COUNTY,0.0,C,3116.84,2001-10-03,210,ROADS AND BRIDGES,0.0,E,O,C,553d4c29-85ef-4002-ad7f-0b10c0b3da19
2,6,1709,0,"SEVERE STORMS, TORNADOES, AND FLOODING",DR,Severe Storm,2007-06-29,TX,Texas,Henderson,213-99213-00,HENDERSON (COUNTY),0.0,C,7211.69,2007-11-09,2594,ROADS AND BRIDGES,0.0,E,O,C,743771d1-7292-4009-afc7-a8258ebc4414
3,4,1549,0,HURRICANE IVAN,DR,Tropical Cyclone,2004-09-15,AL,Alabama,Baldwin,003-57144-00,ORANGE BEACH,0.0,A,129387.21,2005-01-29,1509,DEBRIS REMOVAL,0.0,E,O,C,4c86e4b3-3e25-4b89-904a-26048d4181c1
4,5,1370,0,"SEVERE WINTER STORMS, FLOODING, AND TORNADOES",DR,Flood,2001-05-16,MN,Minnesota,Houston,055-33866-00,LA CRESCENT,0.0,A,11512.50,2001-09-22,1418,DEBRIS REMOVAL,0.0,E,O,C,158efd8c-5c67-418b-8951-6e3ca9be537e


In [203]:
fema_cleaned.dtypes

region                            int64
disasterNumber                    int64
sriaDisaster                      int64
declarationTitle                 object
disasterType                     object
incidentType                     object
declarationDate          datetime64[ns]
stateAbbreviation                object
state                            object
county                           object
applicantId                      object
applicantName                    object
pnpStatus                       float64
damageCategoryCode               object
federalShareObligated           float64
dateObligated            datetime64[ns]
pwNumber                          int64
projectTitle                     object
versionNumber                   float64
eligibilityStatus                object
fundingStatus                    object
paCloseoutStatus                 object
id                               object
dtype: object

In [204]:
# Checking for null values
fema_cleaned.isna().sum().to_frame(name='null_values').query('null_values>0')

,null_values
county,9291
pnpStatus,9288
projectTitle,4775
versionNumber,9288


In [205]:
fema_cleaned = fema_cleaned.assign(
    projectTitle=lambda x: x["projectTitle"].fillna("missing"),
    county=lambda x: x["county"].fillna("missing"),
    pnpStatus=lambda x: x["pnpStatus"].fillna(-1),
    versionNumber=lambda x: x["versionNumber"].fillna(-1)
)

In [206]:
fema_cleaned.shape

(786608, 23)

## <span style="color:pink">Spliting and Saving the Cleaned Data</span>

In [207]:
X = fema_cleaned.drop('federalShareObligated', axis=1)
y = fema_cleaned['federalShareObligated']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [208]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(629286, 22)
(157322, 22)
(629286,)
(157322,)


In [209]:
def save_train_test_csv(
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    y_train: pd.Series,
    y_test: pd.Series,

) -> None:

    BASE_DIR = Path.cwd().parents[0]

    train_data_dir = BASE_DIR / "data" / "cleaned" / "train"
    test_data_dir = BASE_DIR / "data" / "cleaned" / "test"


    X_train.to_csv(train_data_dir / "X_train.csv", index=False)
    X_test.to_csv(test_data_dir / "X_test.csv", index=False)
    y_train.to_csv(train_data_dir / "y_train.csv", index=False, header=True)
    y_test.to_csv(test_data_dir / "y_test.csv", index=False, header=True)

In [210]:
# Saving Clean Train and Test Data
save_train_test_csv(X_train, X_test, y_train, y_test)